## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [10]:
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.layers import Flatten, Dense, Dropout


In [5]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [14]:
batch_size = 256 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 20 # 訓練整個資料集共 30個循環
IMAGE_SIZE = (32, 32)

# 建立 ResNet 模型
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
# x = Dropout(0.25)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(num_classes, activation='softmax', name='softmax')(x)

model = Model(inputs=net.input, outputs=output_layer)
model.summary()

C:\Users\Sanga\AppData\Roaming\Python\Python37\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [16]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-5),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 51s 1ms/sample - loss: 0.6117 - acc: 0.7942 - val_loss: 3.1122 - val_acc: 0.1258
Epoch 2/20
50000/50000 [==============================] - 41s 823us/sample - loss: 0.5141 - acc: 0.8247 - val_loss: 2.8532 - val_acc: 0.2302
Epoch 3/20
50000/50000 [==============================] - 41s 814us/sample - loss: 0.4660 - acc: 0.8400 - val_loss: 1.6571 - val_acc: 0.4974
Epoch 4/20
50000/50000 [==============================] - 40s 809us/sample - loss: 0.4228 - acc: 0.8561 - val_loss: 0.7492 - val_acc: 0.7535
Epoch 5/20
50000/50000 [==============================] - 41s 813us/sample - loss: 0.3864 - acc: 0.8673 - val_loss: 0.6184 - val_acc: 0.7950
Epoch 6/20
50000/50000 [==============================] - 41s 813us/sample - loss: 0.3522 - acc: 0.8795 - val_loss: 0.6117 - val_acc: 0.7997
Epoch 7/20
50000/50000 [==============================] - 42s 833us/sample - loss: 0.3191 - acc: 0.8905 - 